<a href="https://colab.research.google.com/github/hirdeshkumar2407/NLP_Group_Assigment/blob/feature-cluster-the-docs/1_ClusterTheDocuments.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**Cluster The Documents**

In [1]:
from google.colab import userdata


In [3]:
import pandas as pd

df = pd.read_json("hf://datasets/FreedomIntelligence/RAG-Instruct/rag_instruct.json")

In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 40541 entries, 0 to 40540
Data columns (total 3 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   question   40541 non-null  object
 1   answer     40541 non-null  object
 2   documents  40541 non-null  object
dtypes: object(3)
memory usage: 950.3+ KB


In [5]:
df.head(10)

,question,answer,documents
0,Describe the cultural impact and legacy of the...,The 1967 Disney film 'The Jungle Book' has had...,[decided to make the story more straightforwar...
1,Comparing Taiwanese and Chinese education syst...,Taiwanese students performed exceptionally wel...,[the world with 68.5% of Taiwanese high school...
2,What are some key features and applications of...,PyTorch is an open-source machine learning lib...,[Sparrho Sparrho combines human and artificial...
3,What term describes the post-war condition of ...,The term used to describe the state of Germany...,[zones received 1200 kilocalories a day in off...
4,Compare the nutritional value and potential al...,Kiwifruit is known for its vibrant green or go...,"[(oha), ahahara vegetable and many others keep..."
5,What instruction types does Minsky show to be ...,Minsky shows that his machine is Turing comple...,[instructions in the Table. Our example is bas...
6,Do the plays written by Michael Cook often fea...,"Yes, the plays written by Michael Cook often f...",[St. John's area and celebrated its 25th seaso...
7,Identify an entry-level video editing tool tha...,Freemake Video Converter is an entry-level vid...,[and most consumer and professional NLE system...
8,What evidence found in the Clovis group indica...,"The Clovis group contains goethite, a mineral ...",[of Cumberland Ridge and the Husband Hill summ...
9,How do the characteristics of plant cell walls...,Plant cell walls have unique mechanical and ch...,[cellular elongation: as the cell wall is part...
